In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering, KMeans
import numpy as np
from networkx.algorithms.community.quality import modularity
from sklearn.metrics import silhouette_score

In [2]:
# Load Karate Club graph
G = nx.les_miserables_graph()
print(G.number_of_nodes(), G.number_of_edges())

# assign index to each character
mapping = dict(zip(G.nodes(), range(G.number_of_nodes())))
G = nx.relabel_nodes(G, mapping)

77 254


In [3]:
# apply spectral clustering

sc = SpectralClustering(n_clusters=2, affinity='precomputed', n_init=10)
sc.fit(nx.adjacency_matrix(G).todense())
# print the clustering
print(sc.labels_)
km = KMeans(n_clusters=2, random_state=0, n_init=10).fit(nx.adjacency_matrix(G).todense())
print(km.labels_)
clusters1 = [set() for i in range(7)]
clusters2 = [set() for i in range(7)]
for i in range(len(sc.labels_)):
    clusters1[sc.labels_[i]].add(i)
    clusters2[km.labels_[i]].add(i)
# print the modularity score
from networkx.algorithms.community.quality import modularity
print(modularity(G, clusters1))
print(modularity(G, clusters2))

# silhouette score
print(silhouette_score(nx.adjacency_matrix(G).todense(), sc.labels_))
print(silhouette_score(nx.adjacency_matrix(G).todense(), km.labels_))

[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
0.208024241522903
0.09219214753123145
0.04227845964148498
0.6871403653256811


In [ ]:
# MSDR Comparison
# Bigger dataset
# COmpare outputs and find improvs Nashcode.
# another graph clustering and game based clustering

In [4]:
# Using Game Theory to find the best clustering
import math
# Initial Partition
# Before we run the algorithm, we have to start with an initial partition of the given graph. Our choice of initial partition is such that each group in the partition has 3 nodes (with the exception of possibly one group, as n, %3 need not be 0). We choose these 3 nodes per cluster such that they are possibly connected with each other.

def initial_partition(G):
    # get the number of nodes in the graph
    n = G.number_of_nodes()
    # get the number of clusters
    factor = 3
    k = int(n/factor)
    # get the nodes in the graph
    nodes = list(G.nodes())
    # initialize the partition
    partition = []
    # for each cluster
    for i in range(k):
        # get the nodes in the cluster
        cluster = nodes[i*factor:(i+1)*factor]
        # add the cluster to the partition
        partition.append(set(cluster))
    # if there are any nodes left
    if n%3 != 0:
        # add them to the last cluster
        partition[-1].update(nodes[-(n % factor):])
    # return the partition
    return partition
    
    


In [5]:
# Order of Nodes
# We consider the nodes in the order of non-decreasing degree sequence. The reason for this choice is as follows: when a node with small degree jumps to another group, then it creates less damage to the utility of the rest of the nodes than that of a node with high degree.


def order_of_nodes(G):
    # get the degree sequence
    degree_sequence = list(G.degree())
    # sort the degree sequence
    degree_sequence.sort(key=lambda x: x[1], reverse=False)
    # return the order of nodes
    print(degree_sequence)
    return [x[0] for x in degree_sequence]

In [6]:
order = order_of_nodes(G)
print(order)

[(0, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (11, 1), (13, 1), (14, 1), (15, 1), (32, 1), (40, 1), (45, 1), (47, 1), (53, 1), (67, 1), (12, 2), (30, 2), (33, 2), (44, 2), (46, 2), (50, 2), (52, 2), (56, 2), (73, 2), (74, 2), (2, 3), (3, 3), (39, 3), (42, 3), (43, 3), (72, 3), (28, 4), (31, 4), (54, 4), (34, 6), (35, 6), (36, 6), (37, 6), (38, 6), (16, 7), (18, 7), (19, 7), (20, 7), (21, 7), (22, 7), (49, 7), (51, 7), (75, 7), (76, 7), (29, 8), (17, 9), (60, 9), (71, 9), (1, 10), (66, 10), (68, 10), (69, 10), (70, 10), (24, 11), (26, 11), (41, 11), (57, 11), (59, 11), (61, 11), (63, 12), (65, 12), (62, 13), (64, 13), (23, 15), (58, 15), (25, 16), (27, 17), (55, 19), (48, 22), (10, 36)]
[0, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 32, 40, 45, 47, 53, 67, 12, 30, 33, 44, 46, 50, 52, 56, 73, 74, 2, 3, 39, 42, 43, 72, 28, 31, 54, 34, 35, 36, 37, 38, 16, 18, 19, 20, 21, 22, 49, 51, 75, 76, 29, 17, 60, 71, 1, 66, 68, 69, 70, 24, 26, 41, 57, 59, 61, 63, 65, 62, 64, 23, 58, 25, 27, 55, 48,

In [11]:
from networkx.algorithms.distance_measures import eccentricity

def payoff2(i,G,S,partition):
    # di(S)  is the number of neighbors of node i in community S,
    diS = len([n for n in G.neighbors(i) if n in S])
    # Ti(S) is number of pairs of neighbors of node i in S that are connected themselves
    TiS = len([(n1,n2) for n1 in G.neighbors(i) if n1 in S for n2 in G.neighbors(i) if n2 in S and G.has_edge(n1,n2)])
    # Ji(S) is ratio to number of edges within S and incident to node i and the number of edges outside S and incident to node i
    JiS = len([(n1,n2) for n1 in G.neighbors(i) if n1 not in S for n2 in G.neighbors(i) if n2 not in S and G.has_edge(n1,n2)])
    
    u = diS + ((4*TiS**2 - JiS**2)/(diS + 1)) + JiS/(TiS + 1) + (1.0/(eccentricity(G,i) + 1)) + modularity(G,partition)
    return u

In [8]:
def payoff(i,G,S,partition):
    # di(S)  is the number of neighbors of node i in community S,
    diS = len([n for n in G.neighbors(i) if n in S])
    # Ti(S) is number of pairs of neighbors of node i in S that are connected themselves
    TiS = len([(n1,n2) for n1 in G.neighbors(i) if n1 in S for n2 in G.neighbors(i) if n2 in S and G.has_edge(n1,n2)])
    # Ci(S) is the average eccentricity of the nodes in S
    u = (diS + 4*TiS/((diS-1.1)))
    return u

In [12]:
def NASHcode(G, partition, order):
    # get the number of nodes in the graph
    n = G.number_of_nodes()
    # get the number of clusters
    k = len(partition)
    # get the adjacency matrix
    A = nx.adjacency_matrix(G).todense()
    # initialize the utility
    utility = 0
    # for each node
    c = 0
    while True:
        flag = 0
        # print(c)
        for i in range(n):
            #print(i)
            # get the node
            j = order[i]
            # get the cluster of the node
            k = len(partition)
            l = [m for m in range(k) if j in partition[m]][0]
            
            # for each cluster other than the cluster of the node
            for m in range(k):
                if m != l and len(partition[m]) > 0:
                    # get the utility of the node in the cluster
                    u = payoff2(j,G,partition[m],partition)
                    p = payoff2(j,G,partition[l],partition)
                    # if the utility is greater than the utility of the node in its own cluster
                    if u > p:
                        # update the utility
                        utility += u - p
                        # update the partition
                        #print(partition[l],partition[m],j)
                        partition[l].remove(j)
                        partition[m].add(j)
                        flag = 1
                        l = m
            
        if flag == 0:
            break
        c += 1
    
    return partition

In [13]:
clusters = NASHcode(G, initial_partition(G), order_of_nodes(G))
print(modularity(G, clusters))

labels = [0] * G.number_of_nodes()
for i in range(len(clusters)):
    for j in clusters[i]:
        labels[j] = i

print(silhouette_score(nx.adjacency_matrix(G).todense(), labels, metric='cosine'))

[(0, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (11, 1), (13, 1), (14, 1), (15, 1), (32, 1), (40, 1), (45, 1), (47, 1), (53, 1), (67, 1), (12, 2), (30, 2), (33, 2), (44, 2), (46, 2), (50, 2), (52, 2), (56, 2), (73, 2), (74, 2), (2, 3), (3, 3), (39, 3), (42, 3), (43, 3), (72, 3), (28, 4), (31, 4), (54, 4), (34, 6), (35, 6), (36, 6), (37, 6), (38, 6), (16, 7), (18, 7), (19, 7), (20, 7), (21, 7), (22, 7), (49, 7), (51, 7), (75, 7), (76, 7), (29, 8), (17, 9), (60, 9), (71, 9), (1, 10), (66, 10), (68, 10), (69, 10), (70, 10), (24, 11), (26, 11), (41, 11), (57, 11), (59, 11), (61, 11), (63, 12), (65, 12), (62, 13), (64, 13), (23, 15), (58, 15), (25, 16), (27, 17), (55, 19), (48, 22), (10, 36)]
0.5197174301011303
0.33872762120991057


In [2]:
# Read dataset facebook_combined.txt

G = nx.read_edgelist('facebook_combined.txt', create_using=nx.Graph(), nodetype=int)
print(G.number_of_nodes(), G.number_of_edges())


4039 88234


In [86]:
# Kmeans
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=1346, random_state=0, n_init=10).fit(nx.adjacency_matrix(G).todense())

# modularity
clusters = [set() for i in range(1346)]
for i in range(len(kmeans.labels_)):
    clusters[kmeans.labels_[i]].add(i)

print(modularity(G, clusters))


0.03435810999294669


In [268]:
# sihouette score
labels = [0] * G.number_of_nodes()
for i in range(len(clusters)):
    for j in clusters[i]:
        labels[j] = i

print(silhouette_score(nx.adjacency_matrix(G).todense(), labels, metric='cosine'))

-0.006518192364034966


In [271]:
# spectral clustering
from sklearn.cluster import SpectralClustering
sc = SpectralClustering(n_clusters=1346, affinity='precomputed', n_init=10)
sc.fit(nx.adjacency_matrix(G).todense())
clusters = [set() for i in range(1346)]
for i in range(len(sc.labels_)):
    clusters[sc.labels_[i]].add(i)

print(modularity(G, clusters))

In [269]:
# sihouette score
labels = [0] * G.number_of_nodes()
for i in range(len(clusters)):
    for j in clusters[i]:
        labels[j] = i

print(silhouette_score(nx.adjacency_matrix(G).todense(), labels, metric='cosine'))

-0.006518192364034966


In [10]:
order = order_of_nodes(G)
print(order)

[(107, 1045), (1684, 792), (1912, 755), (3437, 547), (0, 347), (2543, 294), (2347, 291), (1888, 254), (1800, 245), (1663, 235), (1352, 234), (2266, 234), (483, 231), (348, 229), (1730, 226), (1985, 224), (1941, 223), (2233, 222), (2142, 221), (1431, 220), (1199, 217), (1584, 211), (2206, 210), (1768, 209), (2229, 207), (2410, 207), (2611, 207), (1086, 205), (1589, 205), (2047, 205), (2218, 205), (2078, 204), (1993, 203), (2123, 203), (1746, 202), (2464, 202), (1827, 201), (2240, 201), (2507, 201), (2560, 201), (2244, 200), (1983, 199), (2309, 199), (1126, 198), (2088, 198), (2131, 198), (2340, 198), (2602, 198), (2324, 197), (2369, 197), (2590, 197), (2542, 196), (2604, 196), (1804, 195), (2073, 195), (2220, 195), (2607, 195), (2188, 194), (1390, 193), (2059, 193), (2172, 193), (1943, 192), (2150, 192), (1833, 191), (1946, 191), (2428, 191), (2526, 191), (1377, 190), (1612, 190), (1917, 190), (2201, 190), (2331, 190), (2601, 190), (1621, 189), (1938, 189), (2090, 189), (2384, 188), (21

In [22]:
labels = [0]*G.number_of_nodes()
clusters = NASHcode(G, initial_partition(G), order_of_nodes(G))

labels = [0] * G.number_of_nodes()
for i in range(len(clusters)):
    for j in clusters[i]:
        labels[j] = i

print(silhouette_score(nx.adjacency_matrix(G).todense(), labels, metric='cosine'))


-0.010055814044583498


In [32]:

clusters = NASHcode(G, clusters, order_of_nodes(G))
print(modularity(G, clusters))

0.821666098179199


In [247]:
# Documet clustering dataset

G = nx.read_edgelist('graph.txt', create_using=nx.Graph(), nodetype=int, data=(('weight',float),))
print(G.number_of_nodes(), G.number_of_edges())
MST = nx.minimum_spanning_tree(G)
print(MST.number_of_nodes(), MST.number_of_edges())

# Kmeans
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0, n_init=10).fit(nx.adjacency_matrix(G).todense())

# modularity
clusters = [set() for i in range(20)]
for i in range(len(kmeans.labels_)):
    clusters[kmeans.labels_[i]].add(i)

print(modularity(G, clusters))

# sihouette score
from sklearn.metrics import silhouette_score
print(silhouette_score(nx.adjacency_matrix(G).todense(), kmeans.labels_))

1000 499044
1000 999
0.011219874987835726
0.13560382035327465


In [248]:
# spectral clustering
from sklearn.cluster import SpectralClustering
sc = SpectralClustering(n_clusters=30, affinity='precomputed', n_init=10)
sc.fit(nx.adjacency_matrix(MST).todense())
clusters = [set() for i in range(30)]
for i in range(len(sc.labels_)):
    clusters[sc.labels_[i]].add(i)

print(modularity(MST, clusters))

# sihouette score
from sklearn.metrics import silhouette_score
print(silhouette_score(nx.adjacency_matrix(G).todense(), sc.labels_, metric='cosine'))

0.7551629323850014
-0.5836097662017544


In [244]:
clusters = NASHcode(MST, initial_partition(MST), order_of_nodes(MST))
print(modularity(MST, clusters))

labels = [0]*G.number_of_nodes()
for i in range(len(clusters)):
    for j in clusters[i]:
        labels[j] = i

# sihouette score
from sklearn.metrics import silhouette_score
print(silhouette_score(nx.adjacency_matrix(G).todense(), labels, metric='cosine'))

[(503, 338), (202, 200), (380, 129), (959, 116), (565, 92), (191, 60), (468, 25), (881, 24), (698, 7), (553, 3), (249, 2), (300, 2), (309, 2), (347, 2), (394, 2), (417, 2), (592, 2), (612, 2), (636, 2), (713, 2), (753, 2), (754, 2), (783, 2), (953, 2), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (

In [242]:
clusters = NASHcode(MST, initial_partition(MST), order_of_nodes(MST))
print(modularity(MST, clusters))

labels = [0]*G.number_of_nodes()
for i in range(len(clusters)):
    for j in clusters[i]:
        labels[j] = i

# sihouette score
from sklearn.metrics import silhouette_score
print(silhouette_score(nx.adjacency_matrix(G).todense(), labels, metric='cosine'))

[(503, 338), (202, 200), (380, 129), (959, 116), (565, 92), (191, 60), (468, 25), (881, 24), (698, 7), (553, 3), (249, 2), (300, 2), (309, 2), (347, 2), (394, 2), (417, 2), (592, 2), (612, 2), (636, 2), (713, 2), (753, 2), (754, 2), (783, 2), (953, 2), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (